In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tik
import pandas as pd
from datetime import datetime, timezone, timedelta

from scipy.stats import norm
import matplotlib.mlab as mlab

from operator import add

#from matplotlib import gridspec
#from scipy.optimize import curve_fit

# Input data and log

In [ ]:
#file path and name
filepath = './rawdata/'
filename = 'Feb18_minangle.dat'

time_category, error, direction, timercount, reference = np.loadtxt(filepath + filename, comments='#', unpack=True)

In [ ]:
logpath = './log/'
logfilename = 'PMX_2021-02-18.dat'

df = pd.read_csv(logpath + logfilename, delim_whitespace=True, comment='#', skiprows=8)

In [ ]:
df

In [ ]:
iteration = 100

list_amp = [3.0]
list_intv = ['0.100', '0.120', '0.140', '0.160', '0.180', '0.200', '0.220', '0.240', '0.260', '0.280', '0.300']

matrix_size = 11

pack_size = 1

In [ ]:
min(error)

## Set variables2

In [ ]:
time = timercount/200e6

plot_slice = 30
capture_slice = 500

pos_roll = 5

casp_threshold = -100

Deg = 360/52000

In [ ]:
storepath = './plots/'
storename = 'Feb18_mindegree'

In [ ]:
#first scketch of the rawdata
fig01, axarr01 = plt.subplots(1,2,figsize=(14,7))
ax01 = axarr01.ravel()

ax01[0].plot(time[::100], reference[::100])
ax01[0].set_xlabel('time[sec]')
ax01[0].set_ylabel('reference count')

ax01[1].scatter(np.diff(timercount), np.diff(reference))
ax01[1].set_xlabel('difference time of lines')
ax01[1].set_ylabel('difference counts of lines')

In [ ]:
np.where((np.diff(timercount) > 110000) | (np.diff(timercount) < 90000))[0]

In [ ]:
hoge = len(np.where((np.diff(timercount) > 110000) | (np.diff(timercount) < 90000))[0])

In [ ]:
for i in range(hoge):
    print(f'{np.diff(timercount)[(np.where((np.diff(timercount) > 110000) | (np.diff(timercount) < 90000))[0])[i]]}')

In [ ]:
plt.scatter(time[::100], reference[::100])
plt.xlim([24000,25250])
plt.ylim([-1,52000])

In [ ]:
#fig01.savefig(storepath + storename + '_RawAndCorr.png')

In [ ]:
def packet_capture(dataframe, list_ampere, list_interval, UnixTime_Data):
    JST = timezone(timedelta(hours=+9), 'JST')
    initial_UnixTime = []
    switching_UnixTime = []
    start_UnixTime = []

    for i in range(iteration):
        start_UnixTime.append([])

    for i in list_ampere:
        for j in list_interval:
            list_datetime = ((dataframe[dataframe.iloc[:,5] == i])[(dataframe[dataframe.iloc[:,5] == i]).iloc[:,8] == j]).iloc[:,0:2].to_numpy()
            for k in range(len(list_datetime)):
                if k%pack_size == 0:
                    start_UnixTime[int(k/pack_size)] = (((datetime.strptime(list_datetime[k][0]+' '+list_datetime[k][1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()
                    initial_UnixTime = np.append(initial_UnixTime, start_UnixTime[int(k/pack_size)])
                    pass
                pass
            switching_UnixTime.append(min(start_UnixTime))
            pass
        pass

    stop_UnixTime = dataframe[dataframe.iloc[:,2] == 'OFF'].iloc[-1,:]
    end_UnixTime = (((datetime.strptime(stop_UnixTime[0]+' '+stop_UnixTime[1],"%Y-%m-%d %H:%M:%S-%Z")).replace(tzinfo=timezone.utc)).astimezone(JST)).timestamp()

    captured_initials = []
    captured_switching = []
    
    for i in range(len(switching_UnixTime)):
        captured_switching.append(max(np.where(time_category[::capture_slice] <= switching_UnixTime[i])[0]))
        pass
    
    for i in range(len(initial_UnixTime)):
        captured_initials.append(max(np.where(time_category[::capture_slice] <= initial_UnixTime[i])[0]))
        pass
    
    captured_end = min(np.where(end_UnixTime + pos_roll <= time_category[::capture_slice])[0])
    
    return captured_switching, captured_initials, captured_end

In [ ]:
swt, inits, end = packet_capture(df, list_amp, list_intv, time_category)

In [ ]:
#block
time_block = []
reference_block = []

for i in range(len(swt)-1):
    time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    reference_block.append(reference[int(np.where(time == (time[::capture_slice])[swt[i]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[swt[i+1]-pos_roll])[0])])
    pass
time_block.append(time[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])
reference_block.append(reference[int(np.where(time == (time[::capture_slice])[swt[-1]-pos_roll])[0]):int(np.where(time == (time[::capture_slice])[end+pos_roll])[0])])

lincount_block = []
block_initials = []
onecycle_time = []
proceeded_degrees = []

for i in range(matrix_size):
    lincount_block.append([])
    lincount_block[i] = np.array(np.zeros(len(reference_block[i])))
    
    count_offset = 0
    casp_flag = 0
    casp = np.where(np.diff(reference_block[i]) <= casp_threshold)[0].tolist()

    for j in range(len(reference_block[i])):
        lincount_block[i][j] = reference_block[i][j]+count_offset
        if casp:
            if j == casp[casp_flag]:
                count_offset += reference_block[i][casp[casp_flag]]
                if len(casp) > casp_flag+1:
                    casp_flag += 1
                    pass
                pass
            pass
        pass
    
    block_initials.append([])
    for j in range(iteration):
        block_initials[i].append(int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*iteration:(i+1)*iteration])[j]-pos_roll])[0]))
        pass
    
    onecycle_time.append([])
    proceeded_degrees.append([])
    for j in range(iteration):
        if j != iteration - 1:
            onecycle_time[i].append(time_block[i][block_initials[i][j+1]]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][block_initials[i][j+1]]-lincount_block[i][block_initials[i][j]])*Deg)
            pass
        else:
            onecycle_time[i].append(time_block[i][-1]-time_block[i][block_initials[i][j]])
            proceeded_degrees[i].append((lincount_block[i][-1]-lincount_block[i][block_initials[i][j]])*Deg)
        pass
    pass

In [ ]:
error_pack = [[2,43], [8,92], [9,33]]

In [ ]:
#avoid error
error_position = []
error_flag0 = 0

error_flag2 = 0

for i in range(matrix_size):
    error_flag1 = 1
    error_position.append([])
    for j in range(iteration):
        if i == error_pack[error_flag0][0]:
            if error_flag1 <= len(error_pack[error_flag0])-1:
                if j == error_pack[error_flag0][error_flag1]:
                    #print(f'{error_pack[error_flag0][0]} {error_pack[error_flag0][error_flag1]}')
                    error_position[i].append([])
                    error_position[i][j] = False
                    error_flag1 += 1
                    error_flag2 = 1
                    pass
                else:
                    error_position[i].append([])
                    error_position[i][j] = True
                    pass
                pass
            else:
                error_position[i].append([])
                error_position[i][j] = True
                pass
            pass
        else:
            error_position[i].append([])
            error_position[i][j] = True
            pass
        pass
    if error_flag2 == 1:
        if error_flag0 != len(error_pack)-1:
            error_flag0 += 1
            error_flag2 = 0
            pass
        pass
    pass

In [ ]:
cut_place = 2.4

In [ ]:
i = 4
for j in range(23,25):
    if error_position[i][j] != False:
        hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
        place = min(np.where(hoge >= cut_place)[0])
        plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice]), label=f'num_{j}')
    plt.legend(fontsize=20)

In [ ]:
ref_threshold = 100

cut_sum = []
for j in range(99):
    hoge = np.where(np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice]) > ref_threshold)[0]
    if hoge:
        cut_sum.append(j)
        pass
cut_sum
        

In [ ]:
fig10, axarr10 = plt.subplots(3,4,figsize=(16,12))
ax10 = axarr10.ravel()

fuga = []

for i in range(matrix_size):
    fuga.append([])
    for j in range(iteration-1):
        fuga[i].append([])
        if error_position[i][j] != False:
            hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= cut_place)[0])
            ax10[i].plot(((time_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j] + place])[::plot_slice]))
            #ax10[i].set_title(f'{round((1.5+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
        

In [ ]:
moved_angle = []
start_position = []

figX, axarrX = plt.subplots(3,4,figsize=(16,12))
plt.rcParams["font.size"] = 10
axX = axarrX.ravel()

for i in range(matrix_size):
    moved_angle.append([])
    start_position.append([])
    for j in range(iteration-1):
        if error_position[i][j] != False:
            hoge = (time_block[i][block_initials[i][j]:block_initials[i][j+1]])-time_block[i][block_initials[i][j]]
            place = min(np.where(hoge >= cut_place)[0])
            #print(f'{place} {block_initials[i][j]}')
            moved_angle[i].append((lincount_block[i][block_initials[i][j] + place]-lincount_block[i][block_initials[i][j]])*Deg)
            start_position[i].append(lincount_block[i][block_initials[i][j]]%52000*Deg)
            pass
        pass
    axX[i].scatter(start_position[i], moved_angle[i], color='b')
    axX[i].set_title(f'3.0A_{round(0.10+0.02*i,2)}sec', fontsize=12)

In [ ]:
figX.savefig(storepath + storename + '_POSvsDeg.png', facecolor='white')

In [ ]:
i = 4
np.where(np.array(moved_angle[i]) > 60)[0]

In [ ]:
moved_angle[4][24]

In [ ]:
figY, axarrY = plt.subplots(3,4,figsize=(16,12))
plt.rcParams["font.size"] = 10
axY = axarrY.ravel()

for i in range(matrix_size):
    axY[i].hist(moved_angle[i], bins=10)
    axY[i].set_title(f'3.0A_{round(0.10+0.02*i,2)}sec', fontsize=12)

In [ ]:
figY.savefig(storepath + storename + '_histDeg.png', facecolor='white')

In [ ]:
movDeg_mean = []
movDeg_stdv = []

for i in range(matrix_size):
    movDeg_mean.append(np.mean(moved_angle[i]))
    movDeg_stdv.append(np.std(moved_angle[i]))
    pass

In [ ]:
figXX = plt.figure(figsize=(10,10))
plt.rcParams["font.size"] = 20

plt.plot(movDeg_mean, movDeg_stdv, label=f'3.0A_0.10~0.30sec', color='r')
plt.scatter(movDeg_mean, movDeg_stdv, color='r')
plt.scatter(movDeg_mean[3], movDeg_stdv[3], color='blue', s=70, label=f'3.0A_0.16sec')

plt.xlim([-0.2,6.2])
plt.ylim([-0.05,1.1])

plt.xlabel('respective mean of movceeded degrees[degree]')
plt.ylabel('standard deviation[degree]')
plt.grid()
plt.legend(loc='lower right', fontsize=25)

In [ ]:
figXX.savefig(storepath + storename + '_DEGvsSTD1.png', transparent=True)

In [ ]:
len(lincount_block[0])

In [ ]:
fig10.savefig(storepath + storename +'_speed1-avoidERROR.png', facecolor='white')

In [ ]:
i = 14
for j in range(32,34):
    #if error_position[i][j] != False:
    plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
                np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), label=f'num_{j}')
    plt.legend()

In [ ]:
i = 14
j = 20
plt.plot(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]-time_block[i][block_initials[i][j]],
        np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]))

In [ ]:
i = 19
j = 20

hoge = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
fuga = np.where(~hoge[:-1] & hoge[1:])
piyo = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:]
kue = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:])[fuga]

figY = plt.figure(figsize=(10,7))

plt.plot(piyo-kue, np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), color='k', label='new black belt')
plt.axvline(0.95)
plt.axvline(1.15)
plt.xlabel('time[sec]')
plt.ylabel('difference of counts (~speed)')
plt.xlim([-0.5,2.])
plt.ylim([-20,330])
plt.legend(loc='upper left', fontsize=15)

In [ ]:
proceeded_degrees[i][j]

In [ ]:
figY.savefig(storepath + storename + '_form_21e-1A-1sec_num20.png', transparent=True)

In [ ]:
i = 19
j = 20

toki = np.diff(((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]))
basho = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])

hoge = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
fuga = np.where(~hoge[:-1] & hoge[1:])
piyo = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1]
kue = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[:-1])[fuga]
saku = basho/toki

figY = plt.figure(figsize=(10,7))

plt.plot((piyo-kue)[1:], np.diff(saku)/np.diff(toki), color='k', label='new black belt')
plt.axvline(0)
plt.xlabel('time[sec]')
plt.ylabel('acceleration')
plt.xlim([0.95,1.15])
plt.ylim([-20,1e9])
plt.legend(loc='upper left', fontsize=15)

* <span style="font-size: 200%; color: red;">Here is a check point of analysis.</span>

In [ ]:
fig11 = plt.figure(figsize=(10,10))

i = 0

plt.scatter(np.array(linCount_woErr[i])%52000*Deg,proDeg_woErr[i])
plt.title('3.0A_0.69sec')

In [ ]:
fig11.savefig(storepath + storename + '_POSvsDEG-avoidERROR.png',facecolor='white')

In [ ]:
fig12 = plt.figure(figsize=(10,10))

i = 0

plt.hist(proDeg_woErr[i])
plt.title('3.0A_0.69sec')

In [ ]:
fig12.savefig(storepath + storename + '_histDEG-avoidERROR.png', facecolor='white')

In [ ]:
proDeg_mean = []
proDeg_stdv = []

for i in range(6):
    proDeg_mean.append([])  
    proDeg_stdv.append([])
for i in range(matrix_size):
    proDeg_mean[int(i/6)].append(np.mean(proDeg_woErr[i]))
    proDeg_stdv[int(i/6)].append(np.std(proDeg_woErr[i]))
    pass

In [ ]:
fig14 = plt.figure(figsize=(10,7))
mu, std = norm.fit(proDeg_woErr)

# Plot the histogram.
plt.hist(proDeg_woErr, bins=15, density=True, alpha=0.3, color='r', label='new black belt:3.0A 0.69sec')
plt.legend(fontsize=15,loc='upper left')

# Plot the PDF.
xmin, xmax = plt.xlim([0,40])
plt.ylim([0.,0.18])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
fig14.savefig(storepath + storename + '_histDEG_FIT1.png', transparent=True)

In [ ]:
i = 0
hoge = 10
fuga = 500

time0 = []
time1 = []
time2 = []
time3 = []

for j in range(hoge,fuga):
    time0.append([])
    time1.append([])
    time2.append([])
    time3.append([])
    time0[j-hoge] = np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]) > 0
    time1[j-hoge] = np.where(~time0[j-hoge][:-1] & time0[j-hoge][1:])
    time2[j-hoge] = ((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:]
    time3[j-hoge] = (((time_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice])[1:])[time1[j-hoge]]

figY = plt.figure(figsize=(10,7))

for j in range(hoge,fuga):
    plt.plot(time2[j-hoge]-time3[j-hoge], np.diff((lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]), color='k', label='black belt')
    pass

plt.axvline(0.65, color='y')
plt.axvline(0.78, color='b')
plt.axhline(60)
plt.xlabel('time[sec]')
plt.ylabel('difference of counts (~speed)')
plt.xlim([-0.5,2.])
plt.ylim([-20,140])
#plt.legend(loc='upper right', fontsize=15)

In [ ]:
figY.savefig(storepath + storename + '_form_3A-69e-1sec_num10_slice30.png', transparent=True)

In [ ]:
kue = (lincount_block[i][block_initials[i][j]:block_initials[i][j+1]])[::plot_slice]

In [ ]:
(kue[-1] - kue[0])*Deg

In [ ]:
figXX = plt.figure(figsize=(10,7))
for i in range(6):
    plt.plot(proDeg_mean[i], proDeg_stdv[i], label=f'{round(1.5+i*0.3,2)}A')
    plt.scatter(proDeg_mean[i], proDeg_stdv[i])
    pass
plt.xlabel('respective mean of proceeded degrees[degree]')
plt.ylabel('standard deviation[degree]')
# plt.xlim([-30,670])
# plt.ylim([0,16])
# plt.axvline(360)
plt.legend(loc='upper left', fontsize=15)

In [ ]:
figXX.savefig(storepath + storename + '_DEGvsSTD0.png', facecolor='white')

In [ ]:
i = 12
plt.plot(time_block[i], lincount_block[i])
plt.axvline(time_block[i][block_initials[i][20]])

In [ ]:
linear_refcount = []

specific_refcount = reference_block[12]

for
linear_refcount = np.array(np.zeros(len(specific_refcount)))
    
count_offset = 0
casp_flag = 0
casp = np.where(np.diff(specific_refcount) <= casp_threshold)[0].tolist()

for i in range(len(specific_refcount)):
    linear_refcount[i] = specific_refcount[i]+count_offset
    if casp:
        if i == casp[casp_flag]:
            count_offset += specific_refcount[casp[casp_flag]]
            if len(casp) > casp_flag+1:
                casp_flag += 1
                pass
            pass
        pass
    pass


In [ ]:
len(inits[0:100])

In [ ]:
plt.plot(time_block[12], linear_refcount)
plt.axvline((time[::capture_slice])[(inits[1200:1300])[90]])

In [ ]:
linear_refcount

In [ ]:
hoge = np.where(time_block[12] == (time[::capture_slice])[(inits[1200:1300])[90]])[0]
hoge

In [ ]:
(time_block[12])[hoge]

In [ ]:
i = 3
hoge = int(np.where(time_block[i] == (time[::capture_slice])[(inits[i*100:(i+1)*100])[j]])[0])

In [ ]:
hoge

In [ ]:
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

In [ ]:
#del df, initial_lines, time, rep, hoge

# Plots

In [ ]:
fig10 ,axarr10 = plt.subplots(6,6,figsize=(26,28))
ax10 = axarr10.ravel()

for i in range(matrix_size):
    for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
        #if error_position[i][j] != False:
        ax10[i].plot((linTime[i][:-1])[int(edge_initial[i][j]):int(edge_final[i][j])]-(linTime[i][:-1])[int(edge_initial[i][j])],
                    (np.diff(linRef[i]))[int(edge_initial[i][j]):int(edge_final[i][j])])
        ax10[i].set_title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
        #ax10[i].xaxis.set_major_locator(tik.MultipleLocator(0.5))

In [ ]:
#fig10.savefig(storepath + storename +'_speed1.png', facecolor='white')

In [ ]:
i = 0

fig = plt.figure(figsize=(10,7))
#for j in range(min(len(edge_initial[i]),len(edge_final[i]))):
for j in range(0,10):
    #if error_position[i][j] != False:
    plt.plot((linTime[i][:-1])[int(edge_initial[i][j]):int(edge_final[i][j])]-(linTime[i][:-1])[int(edge_initial[i][j])],
                (np.diff(linRef[i]))[int(edge_initial[i][j]):int(edge_final[i][j])])
    plt.title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')

In [ ]:
fig11 ,axarr11 = plt.subplots(6,6,figsize=(24,22))
ax11 = axarr11.ravel()

for i in range(matrix_size):
    ax11[i].scatter(linInitRef_woErr[i],proDeg_woErr[i], s=20)
    ax11[i].set_title(f'{round((1.2+0.3*int(i/6)),2)}A_{0.5*(1+i%6)}sec')
    pass

In [ ]:
#fig11.savefig(storepath + storename + '_TvsDEG.png', facecolor='white')

In [ ]:
fig12 = plt.figure(figsize=(10,7))
for i in range(matrix_size):
    plt.scatter(linInitRef_woErr[i], proDeg_woErr[i])
    pass
plt.title('2.1A 0.55sec position VS degrees')
plt.xlabel('position[deg]')
plt.ylabel('proceeded degrees[deg]')

In [ ]:
int(len(linInitRef_woErr[0])/16)

In [ ]:
hoge = []
for i in range(int(len(linInitRef_woErr[0])/16)):
    hoge.append(linInitRef_woErr[0][16*i:16*(i+1)])
    pass
#hoge

In [ ]:
360/23

In [ ]:
plt.hist(linInitRef_woErr[0],range=[0,360],bins=15)

In [ ]:
hist_color = ['bisque', 'lightgreen', 'skyblue']
fuga = []
for i in range(int(len(linInitRef_woErr[0])/16)):
    fuga.append(hist_color[i%3])

In [ ]:
fig = plt.figure(figsize=(10,7))
plt.hist(hoge,range=[0,360],bins=15,stacked=True,color=fuga)

In [ ]:
#fig12.savefig(storepath + storename + '_POSvsDEG.png', facecolor='white')

In [ ]:
fig13 = plt.figure()
for i in range(matrix_size):
    plt.hist(proDeg_woErr[i], bins=15)
    pass
plt.title('2.1A 0.55sec proceeded degrees hist')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
#fig13.savefig(storepath + storename + '_histDEG.png', facecolor='white')

In [ ]:
#Fitting histogram
fig14 = plt.figure(figsize=(10,7))
mu, std = norm.fit(proDeg_woErr)

# Plot the histogram.
plt.hist(proDeg_woErr, bins=15, density=True, alpha=0.3, color='g', label='2.1A 0.55sec')
plt.legend(fontsize=15,loc='upper right')

# Plot the PDF.
xmin, xmax = plt.xlim([0,40])
plt.ylim([0.,0.18])
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
plt.title(f'Fit Result: mu={round(mu,2)}, std={round(std,2)}')
plt.xlabel('proceeded degrees[deg]')
plt.ylabel('number of events')

In [ ]:
#fig14.savefig(storepath + storename + '_histDEG_FIT0.png', facecolor='white')

# Others